# Create Google_model_V1:

In [1]:
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

import os
#os.environ['KERAS_BACKEND'] = 'theano'
import tensorflow as tf
import keras
from keras import backend as K
keras.backend.set_image_data_format('channels_first')
from keras.layers.core import Layer
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
#sess = tf.Session(config=config)
#K.set_session(sess)
#K.set_learning_phase(1)
import imageio
import time
import scipy
import numpy as np
from PIL import Image
import glob
import matplotlib.pyplot as plt
from keras.utils import plot_model

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import ZeroPadding2D, Dropout, Flatten, concatenate, Reshape, Activation, Softmax
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator



/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [2]:
# configration:
image_channels = 3
image_width = 224
image_height = 224
batch_size = 32
num_classes = 10
epochs = 1
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'


# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


In [3]:
# Resizing images (32-->224) and sample 10k from it.
resized_img = []
for i in range(10000):
    resized_img.append(cv2.resize(np.swapaxes(x_train[i],0,-1),(image_width, image_height)))
x_train = np.swapaxes(resized_img,1,-1)

resized_img = []
for i in range(10000):
    resized_img.append(cv2.resize(np.swapaxes(x_test[i],0,-1),(image_width, image_height)))
x_test = np.swapaxes(resized_img,1,-1)

# sample from labels
y_train = y_train[:10000]
y_test = y_test[:10000]

In [4]:
# duplicate labels as we have 3 output heads
temp = []
temp.append(y_train)
temp.append(y_train)
temp.append(y_train)
print(np.swapaxes(np.swapaxes(np.asarray(temp),0,1),1,2).shape)
y_train = np.swapaxes(np.swapaxes(np.asarray(temp),0,1),1,2)

temp = []
temp.append(y_test)
temp.append(y_test)
temp.append(y_test)
print(np.swapaxes(np.swapaxes(np.asarray(temp),0,1),1,2).shape)
y_test = np.swapaxes(np.swapaxes(np.asarray(temp),0,1),1,2)

(10000, 10, 3)
(10000, 10, 3)


In [4]:
class LRN2D(Layer):
    """
    This code is adapted from pylearn2.
    License at: https://github.com/lisa-lab/pylearn2/blob/master/LICENSE.txt
    """

    def __init__(self, alpha=0.0001, k=1, beta=0.75, n=5, **kwargs):
        if n % 2 == 0:
            raise NotImplementedError("LRN2D only works with odd n. n provided: " + str(n))
        super(LRN2D, self).__init__(**kwargs)
        self.alpha = alpha
        self.k = k
        self.beta = beta
        self.n = n

    def call(self, X):
        b, ch, r, c = K.int_shape(X)
        half_n = self.n // 2
        input_sqr = K.square(X)
        if b == None:
          extra_channels = K.zeros((ch + 2 * half_n, r, c), dtype = 'float32')
          extra_channels = K.expand_dims(extra_channels, axis = 0)
        else:
          extra_channels = K.zeros((b, ch + 2 * half_n, r, c), dtype = 'float32')
        input_sqr = K.concatenate([extra_channels[:, :half_n, :, :],
                                   input_sqr,
                                   extra_channels[:, half_n + ch:, :, :]],
                                   axis=1)
        scale = self.k
        norm_alpha = self.alpha / self.n
        for i in range(0, self.n):
            scale += norm_alpha * input_sqr[:, i:i + ch, :, :]
        scale = scale ** self.beta
        X = X / scale
        return X

    def get_config(self):
        config = {"name": self.__class__.__name__,
                  "alpha": self.alpha,
                  "k": self.k,
                  "beta": self.beta,
                  "n": self.n}
        base_config = super(LRN2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
    
class PoolHelper(Layer):
    
    def __init__(self, **kwargs):
        super(PoolHelper, self).__init__(**kwargs)
    
    def call(self, x, mask=None):
        return x[:,:,1:,1:]
    
    def get_config(self):
        config = {}
        base_config = super(PoolHelper, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [5]:
def create_googlenet(trainable = True):

    with tf.Session(config=config) as sess:

        input_image = Input(shape=(image_channels, image_width, image_height))

        conv1_7x7_s2 = Conv2D(64,(7,7),strides=(2,2),padding='same',activation='relu',name='conv1/7x7_s2',kernel_regularizer=l2(0.0002), trainable = trainable)(input_image)

        conv1_zero_pad = ZeroPadding2D(padding=(1, 1), trainable = trainable)(conv1_7x7_s2)

        pool1_helper = PoolHelper()(conv1_zero_pad)

        pool1_3x3_s2 = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid',name='pool1/3x3_s2', trainable = trainable)(pool1_helper)

        pool1_norm1 = LRN2D(name = 'LRN_1')(pool1_3x3_s2)

        conv2_3x3_reduce = Conv2D(64,(1,1),padding='same',activation='relu',name='conv2/3x3_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(pool1_norm1)

        conv2_3x3 = Conv2D(192,(3,3),padding='same',activation='relu',name='conv2/3x3',kernel_regularizer=l2(0.0002), trainable = trainable)(conv2_3x3_reduce)

        conv2_norm2 = LRN2D(name = 'LRN_2')(conv2_3x3)

        conv2_zero_pad = ZeroPadding2D(padding=(1, 1), trainable = trainable)(conv2_norm2)

        pool2_helper = PoolHelper()(conv2_zero_pad)

        pool2_3x3_s2 = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid',name='pool2/3x3_s2', trainable = trainable)(pool2_helper)

        inception_3a_1x1 = Conv2D(64,(1,1),padding='same',activation='relu',name='inception_3a/1x1',kernel_regularizer=l2(0.0002), trainable = trainable)(pool2_3x3_s2)

        inception_3a_3x3_reduce = Conv2D(96,(1,1),padding='same',activation='relu',name='inception_3a/3x3_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(pool2_3x3_s2)

        inception_3a_3x3 = Conv2D(128,(3,3),padding='same',activation='relu',name='inception_3a/3x3',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_3a_3x3_reduce)

        inception_3a_5x5_reduce = Conv2D(16,(1,1),padding='same',activation='relu',name='inception_3a/5x5_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(pool2_3x3_s2)

        inception_3a_5x5 = Conv2D(32,(5,5),padding='same',activation='relu',name='inception_3a/5x5',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_3a_5x5_reduce)

        inception_3a_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same',name='inception_3a/pool', trainable = trainable)(pool2_3x3_s2)

        inception_3a_pool_proj = Conv2D(32,(1,1),padding='same',activation='relu',name='inception_3a/pool_proj',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_3a_pool)

        inception_3a_output = concatenate([inception_3a_1x1,inception_3a_3x3,inception_3a_5x5,inception_3a_pool_proj],axis=1)

        inception_3b_1x1 = Conv2D(128,(1,1),padding='same',activation='relu',name='inception_3b/1x1',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_3a_output)

        inception_3b_3x3_reduce = Conv2D(128,(1,1),padding='same',activation='relu',name='inception_3b/3x3_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_3a_output)

        inception_3b_3x3 = Conv2D(192,(3,3),padding='same',activation='relu',name='inception_3b/3x3',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_3b_3x3_reduce)

        inception_3b_5x5_reduce = Conv2D(32,(1,1),padding='same',activation='relu',name='inception_3b/5x5_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_3a_output)

        inception_3b_5x5 = Conv2D(96,(5,5),padding='same',activation='relu',name='inception_3b/5x5',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_3b_5x5_reduce)

        inception_3b_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same',name='inception_3b/pool', trainable = trainable)(inception_3a_output)

        inception_3b_pool_proj = Conv2D(64,(1,1),padding='same',activation='relu',name='inception_3b/pool_proj',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_3b_pool)

        inception_3b_output = concatenate([inception_3b_1x1,inception_3b_3x3,inception_3b_5x5,inception_3b_pool_proj],axis=1)

        inception_3b_output_zero_pad = ZeroPadding2D(padding=(1, 1), trainable = trainable)(inception_3b_output)

        pool3_helper = PoolHelper()(inception_3b_output_zero_pad)

        pool3_3x3_s2 = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid',name='pool3/3x3_s2', trainable = trainable)(pool3_helper)

        inception_4a_1x1 = Conv2D(192,(1,1),padding='same',activation='relu',name='inception_4a/1x1',kernel_regularizer=l2(0.0002), trainable = trainable)(pool3_3x3_s2)

        inception_4a_3x3_reduce = Conv2D(96,(1,1),padding='same',activation='relu',name='inception_4a/3x3_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(pool3_3x3_s2)

        inception_4a_3x3 = Conv2D(208,(3,3),padding='same',activation='relu',name='inception_4a/3x3',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4a_3x3_reduce)

        inception_4a_5x5_reduce = Conv2D(16,(1,1),padding='same',activation='relu',name='inception_4a/5x5_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(pool3_3x3_s2)

        inception_4a_5x5 = Conv2D(48,(5,5),padding='same',activation='relu',name='inception_4a/5x5',kernel_regularizer=l2(0.0002))(inception_4a_5x5_reduce)

        inception_4a_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same',name='inception_4a/pool', trainable = trainable)(pool3_3x3_s2)

        inception_4a_pool_proj = Conv2D(64,(1,1),padding='same',activation='relu',name='inception_4a/pool_proj',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4a_pool)

        inception_4a_output = concatenate([inception_4a_1x1,inception_4a_3x3,inception_4a_5x5,inception_4a_pool_proj],axis=1)

        loss1_ave_pool = AveragePooling2D(pool_size=(5,5),strides=(3,3),name='loss1/ave_pool', trainable = trainable)(inception_4a_output)

        loss1_conv = Conv2D(128,(1,1),padding='same',activation='relu',name='loss1/conv',kernel_regularizer=l2(0.0002), trainable = trainable)(loss1_ave_pool)

        loss1_flat = Flatten()(loss1_conv)

        loss1_fc = Dense(1024,activation='relu',name='loss1/fc',kernel_regularizer=l2(0.0002), trainable = trainable)(loss1_flat)

        loss1_drop_fc = Dropout(0.7, trainable = trainable)(loss1_fc)

        loss1_classifier = Dense(1000, name='loss1/classifier',kernel_regularizer=l2(0.0002), trainable = trainable)(loss1_drop_fc)

        loss1_classifier_act = Activation('softmax', name = 'head1')(loss1_classifier)


        inception_4b_1x1 = Conv2D(160,(1,1),padding='same',activation='relu',name='inception_4b/1x1',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4a_output)

        inception_4b_3x3_reduce = Conv2D(112,(1,1),padding='same',activation='relu',name='inception_4b/3x3_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4a_output)

        inception_4b_3x3 = Conv2D(224,(3,3),padding='same',activation='relu',name='inception_4b/3x3',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4b_3x3_reduce)

        inception_4b_5x5_reduce = Conv2D(24,(1,1),padding='same',activation='relu',name='inception_4b/5x5_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4a_output)

        inception_4b_5x5 = Conv2D(64,(5,5),padding='same',activation='relu',name='inception_4b/5x5',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4b_5x5_reduce)

        inception_4b_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same',name='inception_4b/pool', trainable = trainable)(inception_4a_output)

        inception_4b_pool_proj = Conv2D(64,(1,1),padding='same',activation='relu',name='inception_4b/pool_proj',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4b_pool)

        inception_4b_output = concatenate([inception_4b_1x1,inception_4b_3x3,inception_4b_5x5,inception_4b_pool_proj],axis=1)


        inception_4c_1x1 = Conv2D(128,(1,1),padding='same',activation='relu',name='inception_4c/1x1',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4b_output)

        inception_4c_3x3_reduce = Conv2D(128,(1,1),padding='same',activation='relu',name='inception_4c/3x3_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4b_output)

        inception_4c_3x3 = Conv2D(256,(3,3),padding='same',activation='relu',name='inception_4c/3x3',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4c_3x3_reduce)

        inception_4c_5x5_reduce = Conv2D(24,(1,1),padding='same',activation='relu',name='inception_4c/5x5_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4b_output)

        inception_4c_5x5 = Conv2D(64,(5,5),padding='same',activation='relu',name='inception_4c/5x5',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4c_5x5_reduce)

        inception_4c_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same',name='inception_4c/pool', trainable = trainable)(inception_4b_output)

        inception_4c_pool_proj = Conv2D(64,(1,1),padding='same',activation='relu',name='inception_4c/pool_proj',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4c_pool)

        inception_4c_output = concatenate([inception_4c_1x1,inception_4c_3x3,inception_4c_5x5,inception_4c_pool_proj],axis=1)


        inception_4d_1x1 = Conv2D(112,(1,1),padding='same',activation='relu',name='inception_4d/1x1',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4c_output)

        inception_4d_3x3_reduce = Conv2D(144,(1,1),padding='same',activation='relu',name='inception_4d/3x3_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4c_output)

        inception_4d_3x3 = Conv2D(288,(3,3),padding='same',activation='relu',name='inception_4d/3x3',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4d_3x3_reduce)

        inception_4d_5x5_reduce = Conv2D(32,(1,1),padding='same',activation='relu',name='inception_4d/5x5_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4c_output)

        inception_4d_5x5 = Conv2D(64,(5,5),padding='same',activation='relu',name='inception_4d/5x5',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4d_5x5_reduce)

        inception_4d_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same',name='inception_4d/pool', trainable = trainable)(inception_4c_output)

        inception_4d_pool_proj = Conv2D(64,(1,1),padding='same',activation='relu',name='inception_4d/pool_proj',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4d_pool)

        inception_4d_output = concatenate([inception_4d_1x1,inception_4d_3x3,inception_4d_5x5,inception_4d_pool_proj],axis=1)


        loss2_ave_pool = AveragePooling2D(pool_size=(5,5),strides=(3,3),name='loss2/ave_pool', trainable = trainable)(inception_4d_output)

        loss2_conv = Conv2D(128,(1,1),padding='same',activation='relu',name='loss2/conv',kernel_regularizer=l2(0.0002), trainable = trainable)(loss2_ave_pool)

        loss2_flat = Flatten()(loss2_conv)

        loss2_fc = Dense(1024,activation='relu',name='loss2/fc',kernel_regularizer=l2(0.0002), trainable = trainable)(loss2_flat)

        loss2_drop_fc = Dropout(0.7, trainable = trainable)(loss2_fc)

        loss2_classifier = Dense(1000, name='loss2/classifier',kernel_regularizer=l2(0.0002), trainable = trainable)(loss2_drop_fc)

        loss2_classifier_act = Activation('softmax', name = 'head2')(loss2_classifier)


        inception_4e_1x1 = Conv2D(256,(1,1),padding='same',activation='relu',name='inception_4e/1x1',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4d_output)

        inception_4e_3x3_reduce = Conv2D(160,(1,1),padding='same',activation='relu',name='inception_4e/3x3_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4d_output)

        inception_4e_3x3 = Conv2D(320,(3,3),padding='same',activation='relu',name='inception_4e/3x3',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4e_3x3_reduce)

        inception_4e_5x5_reduce = Conv2D(32,(1,1),padding='same',activation='relu',name='inception_4e/5x5_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4d_output)

        inception_4e_5x5 = Conv2D(128,(5,5),padding='same',activation='relu',name='inception_4e/5x5',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4e_5x5_reduce)

        inception_4e_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same',name='inception_4e/pool', trainable = trainable)(inception_4d_output)

        inception_4e_pool_proj = Conv2D(128,(1,1),padding='same',activation='relu',name='inception_4e/pool_proj',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_4e_pool)

        inception_4e_output = concatenate([inception_4e_1x1,inception_4e_3x3,inception_4e_5x5,inception_4e_pool_proj],axis=1)


        inception_4e_output_zero_pad = ZeroPadding2D(padding=(1, 1), trainable = trainable)(inception_4e_output)

        pool4_helper = PoolHelper()(inception_4e_output_zero_pad)

        pool4_3x3_s2 = MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='valid',name='pool4/3x3_s2', trainable = trainable)(pool4_helper)


        inception_5a_1x1 = Conv2D(256,(1,1),padding='same',activation='relu',name='inception_5a/1x1',kernel_regularizer=l2(0.0002), trainable = trainable)(pool4_3x3_s2)

        inception_5a_3x3_reduce = Conv2D(160,(1,1),padding='same',activation='relu',name='inception_5a/3x3_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(pool4_3x3_s2)

        inception_5a_3x3 = Conv2D(320,(3,3),padding='same',activation='relu',name='inception_5a/3x3',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_5a_3x3_reduce)

        inception_5a_5x5_reduce = Conv2D(32,(1,1),padding='same',activation='relu',name='inception_5a/5x5_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(pool4_3x3_s2)

        inception_5a_5x5 = Conv2D(128,(5,5),padding='same',activation='relu',name='inception_5a/5x5',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_5a_5x5_reduce)

        inception_5a_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same',name='inception_5a/pool', trainable = trainable)(pool4_3x3_s2)

        inception_5a_pool_proj = Conv2D(128,(1,1),padding='same',activation='relu',name='inception_5a/pool_proj',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_5a_pool)

        inception_5a_output = concatenate([inception_5a_1x1,inception_5a_3x3,inception_5a_5x5,inception_5a_pool_proj],axis=1)


        inception_5b_1x1 = Conv2D(384,(1,1),padding='same',activation='relu',name='inception_5b/1x1',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_5a_output)

        inception_5b_3x3_reduce = Conv2D(192,(1,1),padding='same',activation='relu',name='inception_5b/3x3_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_5a_output)

        inception_5b_3x3 = Conv2D(384,(3,3),padding='same',activation='relu',name='inception_5b/3x3',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_5b_3x3_reduce)

        inception_5b_5x5_reduce = Conv2D(48,(1,1),padding='same',activation='relu',name='inception_5b/5x5_reduce',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_5a_output)

        inception_5b_5x5 = Conv2D(128,(5,5),padding='same',activation='relu',name='inception_5b/5x5',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_5b_5x5_reduce)

        inception_5b_pool = MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same',name='inception_5b/pool', trainable = trainable)(inception_5a_output)

        inception_5b_pool_proj = Conv2D(128,(1,1),padding='same',activation='relu',name='inception_5b/pool_proj',kernel_regularizer=l2(0.0002), trainable = trainable)(inception_5b_pool)

        inception_5b_output = concatenate([inception_5b_1x1,inception_5b_3x3,inception_5b_5x5,inception_5b_pool_proj],axis=1)


        pool5_7x7_s1 = AveragePooling2D(pool_size=(7,7),strides=(1,1),name='pool5/7x7_s2', trainable = trainable)(inception_5b_output)

        loss3_flat = Flatten()(pool5_7x7_s1)

        pool5_drop_7x7_s1 = Dropout(0.4, trainable = trainable)(loss3_flat)

        loss3_classifier = Dense(1000, name='loss3/classifier',kernel_regularizer=l2(0.0002), trainable = trainable)(pool5_drop_7x7_s1)

        loss3_classifier_act = Activation('softmax', name='prob')(loss3_classifier)


        googlenet = Model(inputs = input_image, outputs = [loss1_classifier_act, loss2_classifier_act, loss3_classifier_act])
        

        return googlenet

# Training The Model:

In [6]:
data_augmentation = True

In [7]:
model = create_googlenet(trainable = True)
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', loss_weights = [0.3, 0.3, 1], metrics=['accuracy'])

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, [y_train,y_train,y_train], batch_size=batch_size),
                        steps_per_epoch = 10000/batch_size,
                        epochs=epochs,
                        validation_data=(x_test, [y_test,y_test,y_test]),
                        workers=4)

Using real-time data augmentation.


ValueError: `x` (images tensor) and `y` (labels) should have the same length. Found: x.shape = (10000, 3, 224, 224), y.shape = (3, 10000, 10)

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# Testing Model on one image:

In [25]:
#Predicting and testing the pretrained model
master_model = create_googlenet(trainable = False)
master_model.load_weights('googlenet_weights_TF.h5')
#img = scipy.misc.imresize(imageio.imread('/content/gdrive/My Drive/OneLab_Work/Copy of candel.jpeg'), [224, 224]).astype(np.float32)
img = cv2.resize(cv2.imread('TEST4.jpg'), (224, 224)).astype(np.float32)
print(img.shape)
img[:, :, 0] -= 123.68
img[:, :, 1] -= 116.77
img[:, :, 2] -= 103.939
img[:,:,[0,1,2]] = img[:,:,[2,1,0]]
img = img.transpose((2, 0, 1))
img = np.expand_dims(img, axis=0)
test = np.zeros((32, 3, 224, 224))
test[0] = img
print(img.shape)
start = time.time()
out = master_model.predict(img) # note: the model has three outputs
print(time.time()-start)
print('Predicted:', decode_predictions(out[2]))


(224, 224, 3)
(1, 3, 224, 224)
0.7746236324310303
Predicted: [[('n02127052', 'lynx', 0.43880922), ('n02123597', 'Siamese_cat', 0.12934858), ('n02328150', 'Angora', 0.12507504), ('n02123394', 'Persian_cat', 0.11460775), ('n02120079', 'Arctic_fox', 0.06618385)]]


In [12]:
#master_model = create_googlenet('googlenet_weights_TF.h5', trainable = False)
#slave_model = create_googlenet(trainable = True)

#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
#slave_model.compile(optimizer=sgd, loss='categorical_crossentropy', loss_weights = [0.3, 0.3, 1])

In [17]:
layer_names = ["conv1/7x7_s2",
"conv2/3x3_reduce",
"conv2/3x3",
"inception_3a/1x1",
"inception_3a/3x3_reduce",
"inception_3a/3x3",
"inception_3a/5x5_reduce",
"inception_3a/5x5",
"inception_3a/pool_proj",
"inception_3b/1x1",
"inception_3b/3x3_reduce",
"inception_3b/3x3",
"inception_3b/5x5_reduce",
"inception_3b/5x5",
"inception_3b/pool_proj",
"inception_4a/1x1",
"inception_4a/3x3_reduce",
"inception_4a/3x3",
"inception_4a/5x5_reduce",
"inception_4a/5x5",
"inception_4a/pool_proj",
"loss1/conv",
"loss1/fc",
"loss1/classifier"]

In [10]:
def data_generator():
    images = glob.glob("D:/GPU_User3/Dataset/All_Images/*.jpeg")
    batch = np.zeros((32, 3, 224, 224))
    normalization = np.zeros((224, 224, 3))
    normalization[:, :, 0] = 123.68
    normalization[:, :, 1] = 116.779
    normalization[:, :, 2] = 103.939
    labels = np.zeros((32, 1000, 3), dtype = np.uint8)
    model_output = np.zeros((32, 1000, 3), dtype = np.float32)
    #garbage = []

    while 1:
        for img in range(0, 32):
            frame = np.asarray((Image.fromarray(imageio.imread(images[img])).resize((224, 224)).convert("RGB")), dtype = np.float32) - normalization

            frame[:, :, [0, 1, 2]] = frame[:, :, [2, 1, 0]]
            batch[img] = frame.transpose((2, 0, 1))

        labels_temp = np.asarray(master_model.predict(batch)).transpose((1, 2, 0))
        truth = np.max(labels[:, :, -1])
        labels[:, labels_temp == truth, :] = 1
        model_output = np.asarray(master_model.predict(batch)).transpose((1, 2, 0))
        truth_location = np.argmax(model_output[:, :, -1], axis = 1)
        for i in range(0, 32): labels[i, truth_location[i], :] = 1
        
        yield batch, [labels[:, :, 0], labels[:, :, 1], labels[:, :, 2]]

In [11]:
with tf.Session() as sess:
    slave_model = create_googlenet('googlenet_weights_TF.h5', trainable = True)
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    slave_model.compile(optimizer=sgd, loss='categorical_crossentropy', loss_weights = {"prob": 1,
                                                                                        "head2": 0.3,
                                                                                        "head1": 0.3})

    master_model = create_googlenet('googlenet_weights_TF.h5', trainable = False)

OK
OK


In [12]:
slave_model.summary()
plot_model(slave_model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 224, 224)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 64, 112, 112) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 64, 114, 114) 0           conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
pool_helper_1 (PoolHelper)      (None, 64, 114, 114) 0           zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
pool1/3x3_

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [13]:
with tf.Session() as sess:
    
    slave_model = create_googlenet('googlenet_weights_TF.h5', trainable = True)
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    slave_model.compile(optimizer=sgd, loss='categorical_crossentropy', loss_weights = {"prob": 1,
                                                                                    "head2": 0.3,
                                                                                    "head1": 0.3})

    master_model = create_googlenet('googlenet_weights_TF.h5', trainable = False)
    
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    for index in range(0, 5):
        for percentage in range(0, 60, 10):
            weights = slave_model.get_layer(name = layer_names[index]).get_weights()
            weights_flattened = np.concatenate((np.ndarray.flatten(weights[0]), np.ndarray.flatten(weights[1])))

            sorted_weights = np.sort(np.abs(weights_flattened))

            sorted_weights[int(percentage/100*weights_flattened.shape[0]):] = 99999

            #weights_recovered = weights_flattened[0:np.ndarray.flatten(weights[0]).shape[0]].reshape(weights[0].shape)

            pruned_weights_indices1 = np.intersect1d(np.ndarray.flatten(weights[0]), sorted_weights, return_indices = True)[1]

            pruned_weights1 = np.ndarray.flatten(weights[0])
            pruned_weights1[pruned_weights_indices1] = 0.0
            pruned_weights1 = pruned_weights1.reshape(weights[0].shape)


            pruned_weights_indices2 = np.intersect1d(np.abs(np.ndarray.flatten(weights[1])), sorted_weights, return_indices = True)[1]

            pruned_weights2 = np.ndarray.flatten(weights[1])
            pruned_weights2[pruned_weights_indices2] = 0.0
            pruned_weights2 = pruned_weights2.reshape(weights[1].shape)

            pruned_weights_layer = [pruned_weights1, pruned_weights2]

            slave_model.get_layer(name = layer_names[index]).set_weights(pruned_weights_layer)

            print("Prunning The " + str(index) + " Layer at " + str(percentage) + "%\n")

            slave_model.fit_generator(data_generator(),
                                      steps_per_epoch = 14267,
                                      epochs = 10,
                                      verbose = 1,
                                      max_queue_size = 100,
                                      shuffle = True)


OK
OK
Prunning The 0 Layer at 0%

Epoch 1/10


IndexError: list index out of range

In [ ]:
#from theano import function, config, shared, tensor
import numpy
import time

import os
os.environ['THEANO_FLAGS'] = "device=cuda0,force_device=True,floatX=float32"
import theano
from theano import function, config, shared, tensor


vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], tensor.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, tensor.Elemwise) and
              ('Gpu' not in type(x.op).__name__)
              for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')